# 🔬 Machine Learning Klasifikasi Sederhana dengan EDA

**Dataset:** Kondisi Jembatan (Sintesis)

**Tujuan:** Memprediksi kondisi jembatan (Baik/Sedang/Buruk) berdasarkan fitur-fitur inspeksi

---

## 1. Import Libraries

In [ ]:
# Library dasar
import pandas as pd
import numpy as np

# Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Settings
import warnings
warnings.filterwarnings('ignore')

# Style visualisasi
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Libraries berhasil di-import!")

---
## 2. Membuat Dataset Sintesis

In [ ]:
# Set seed untuk reproducibility
np.random.seed(42)
n_samples = 500

# Generate data fitur
data = {
    'usia_jembatan': np.random.randint(5, 60, n_samples),      # tahun
    'panjang': np.random.uniform(10, 200, n_samples),          # meter
    'lebar': np.random.uniform(5, 25, n_samples),              # meter
    'beban_harian': np.random.randint(100, 5000, n_samples),   # kendaraan/hari
    'curah_hujan': np.random.uniform(500, 3000, n_samples),    # mm/tahun
    'tingkat_korosi': np.random.uniform(0, 10, n_samples),     # skala 0-10
    'retakan': np.random.randint(0, 50, n_samples),            # jumlah
    'getaran_max': np.random.uniform(0.1, 5, n_samples),       # mm/s
}

df = pd.DataFrame(data)
print(f"✅ Data fitur dibuat: {df.shape}")

In [ ]:
# Fungsi untuk menentukan kondisi jembatan berdasarkan logika
def tentukan_kondisi(row):
    skor = 0
    
    # Usia jembatan
    if row['usia_jembatan'] > 40: skor += 2
    elif row['usia_jembatan'] > 25: skor += 1
    
    # Tingkat korosi
    if row['tingkat_korosi'] > 7: skor += 2
    elif row['tingkat_korosi'] > 4: skor += 1
    
    # Jumlah retakan
    if row['retakan'] > 30: skor += 2
    elif row['retakan'] > 15: skor += 1
    
    # Getaran maksimum
    if row['getaran_max'] > 3: skor += 1
    
    # Beban harian
    if row['beban_harian'] > 3000: skor += 1
    
    # Tambah noise untuk variasi
    skor += np.random.randint(-1, 2)
    
    # Klasifikasi
    if skor >= 5:
        return 'Buruk'
    elif skor >= 3:
        return 'Sedang'
    else:
        return 'Baik'

# Terapkan fungsi
df['kondisi'] = df.apply(tentukan_kondisi, axis=1)
print(f"✅ Label target 'kondisi' berhasil dibuat!")

In [ ]:
# Simpan dataset
df.to_csv('dataset_jembatan.csv', index=False)
print("✅ Dataset disimpan ke 'dataset_jembatan.csv'")

# Tampilkan sample data
df.head(10)

---
## 3. Exploratory Data Analysis (EDA)

### 3.1 Informasi Dasar Dataset

In [ ]:
# Shape dataset
print(f"📊 Shape Dataset: {df.shape}")
print(f"   - Jumlah baris: {df.shape[0]}")
print(f"   - Jumlah kolom: {df.shape[1]}")

In [ ]:
# Info dataset
print("📋 Informasi Dataset:")
print("=" * 50)
df.info()

In [ ]:
# Tipe data
print("📝 Tipe Data per Kolom:")
print("=" * 50)
for col in df.columns:
    print(f"  {col:20} : {df[col].dtype}")

### 3.2 Statistik Deskriptif

In [ ]:
# Statistik deskriptif untuk fitur numerik
print("📈 Statistik Deskriptif:")
df.describe().round(2)

In [ ]:
# Statistik tambahan
print("📊 Statistik Tambahan:")
print("=" * 60)
numeric_cols = df.select_dtypes(include=[np.number]).columns

stats_df = pd.DataFrame({
    'Mean': df[numeric_cols].mean(),
    'Median': df[numeric_cols].median(),
    'Std': df[numeric_cols].std(),
    'Min': df[numeric_cols].min(),
    'Max': df[numeric_cols].max(),
    'Range': df[numeric_cols].max() - df[numeric_cols].min(),
    'Skewness': df[numeric_cols].skew(),
    'Kurtosis': df[numeric_cols].kurtosis()
}).round(3)

stats_df

### 3.3 Missing Values

In [ ]:
# Cek missing values
print("🔍 Analisis Missing Values:")
print("=" * 50)

missing_df = pd.DataFrame({
    'Kolom': df.columns,
    'Missing Count': df.isnull().sum().values,
    'Missing (%)': (df.isnull().sum().values / len(df) * 100).round(2)
})

print(missing_df.to_string(index=False))
print(f"\n✅ Total missing values: {df.isnull().sum().sum()}")

### 3.4 Distribusi Target Variable

In [ ]:
# Distribusi kelas target
print("🎯 Distribusi Kelas Target:")
print("=" * 50)

target_dist = pd.DataFrame({
    'Kondisi': df['kondisi'].value_counts().index,
    'Jumlah': df['kondisi'].value_counts().values,
    'Persentase (%)': (df['kondisi'].value_counts(normalize=True).values * 100).round(2)
})

print(target_dist.to_string(index=False))

In [ ]:
# Visualisasi distribusi target
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart
colors = ['#2ecc71', '#f39c12', '#e74c3c']
order = ['Baik', 'Sedang', 'Buruk']
ax1 = axes[0]
df['kondisi'].value_counts()[order].plot(kind='bar', ax=ax1, color=colors, edgecolor='black')
ax1.set_title('Distribusi Kondisi Jembatan', fontsize=12, fontweight='bold')
ax1.set_xlabel('Kondisi')
ax1.set_ylabel('Jumlah')
ax1.tick_params(axis='x', rotation=0)

# Tambah label di atas bar
for i, (idx, val) in enumerate(df['kondisi'].value_counts()[order].items()):
    ax1.text(i, val + 5, str(val), ha='center', fontweight='bold')

# Pie chart
ax2 = axes[1]
df['kondisi'].value_counts()[order].plot(kind='pie', ax=ax2, colors=colors, 
                                          autopct='%1.1f%%', startangle=90,
                                          explode=(0.02, 0.02, 0.02))
ax2.set_title('Proporsi Kondisi Jembatan', fontsize=12, fontweight='bold')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

### 3.5 Distribusi Fitur Numerik

In [ ]:
# Histogram untuk semua fitur numerik
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    df[col].hist(bins=25, ax=ax, color='#3498db', edgecolor='black', alpha=0.7)
    ax.set_title(f'Distribusi {col}', fontsize=10, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Frekuensi')
    
    # Tambah garis mean dan median
    ax.axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.2f}')
    ax.axvline(df[col].median(), color='green', linestyle=':', label=f'Median: {df[col].median():.2f}')
    ax.legend(fontsize=7)

plt.suptitle('Distribusi Semua Fitur Numerik', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.6 Boxplot per Kondisi

In [ ]:
# Boxplot untuk setiap fitur per kondisi
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

order = ['Baik', 'Sedang', 'Buruk']
palette = {'Baik': '#2ecc71', 'Sedang': '#f39c12', 'Buruk': '#e74c3c'}

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    sns.boxplot(x='kondisi', y=col, data=df, ax=ax, order=order, palette=palette)
    ax.set_title(f'{col} per Kondisi', fontsize=10, fontweight='bold')
    ax.set_xlabel('')

plt.suptitle('Boxplot Fitur per Kondisi Jembatan', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.7 Analisis Korelasi

In [ ]:
# Correlation matrix
corr_matrix = df[numeric_cols].corr()

# Heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', mask=mask, square=True,
            linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Fitur Numerik', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Korelasi dengan target (encoded)
print("🔗 Korelasi Fitur dengan Target:")
print("=" * 50)

le_temp = LabelEncoder()
df['kondisi_encoded'] = le_temp.fit_transform(df['kondisi'])

correlations = df[numeric_cols.tolist() + ['kondisi_encoded']].corr()['kondisi_encoded'].drop('kondisi_encoded')
correlations_sorted = correlations.sort_values(ascending=False)

print("\nKorelasi (dari tertinggi ke terendah):")
for feat, corr in correlations_sorted.items():
    print(f"  {feat:20} : {corr:+.4f}")

# Hapus kolom temporary
df.drop('kondisi_encoded', axis=1, inplace=True)

In [ ]:
# Visualisasi korelasi dengan target
plt.figure(figsize=(10, 5))
colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in correlations_sorted.values]
correlations_sorted.plot(kind='barh', color=colors, edgecolor='black')
plt.title('Korelasi Fitur dengan Target (Kondisi)', fontsize=12, fontweight='bold')
plt.xlabel('Korelasi')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

### 3.8 Scatter Plot Fitur Penting

In [ ]:
# Scatter plot untuk fitur-fitur penting
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

palette = {'Baik': '#2ecc71', 'Sedang': '#f39c12', 'Buruk': '#e74c3c'}

# Plot 1: Usia vs Tingkat Korosi
sns.scatterplot(data=df, x='usia_jembatan', y='tingkat_korosi', hue='kondisi',
                palette=palette, ax=axes[0], alpha=0.6, s=50)
axes[0].set_title('Usia vs Tingkat Korosi', fontweight='bold')

# Plot 2: Tingkat Korosi vs Retakan
sns.scatterplot(data=df, x='tingkat_korosi', y='retakan', hue='kondisi',
                palette=palette, ax=axes[1], alpha=0.6, s=50)
axes[1].set_title('Tingkat Korosi vs Retakan', fontweight='bold')

# Plot 3: Beban Harian vs Getaran Max
sns.scatterplot(data=df, x='beban_harian', y='getaran_max', hue='kondisi',
                palette=palette, ax=axes[2], alpha=0.6, s=50)
axes[2].set_title('Beban Harian vs Getaran Max', fontweight='bold')

plt.tight_layout()
plt.show()

### 3.9 Pairplot

In [ ]:
# Pairplot untuk fitur terpilih
selected_features = ['usia_jembatan', 'tingkat_korosi', 'retakan', 'getaran_max', 'kondisi']

sns.pairplot(df[selected_features], hue='kondisi', 
             palette=palette, diag_kind='kde',
             plot_kws={'alpha': 0.5, 's': 30})
plt.suptitle('Pairplot Fitur Terpilih', y=1.02, fontsize=14, fontweight='bold')
plt.show()

---
## 4. Data Preprocessing

### 4.1 Pisahkan Features dan Target

In [ ]:
# Pisahkan features (X) dan target (y)
X = df.drop('kondisi', axis=1)
y = df['kondisi']

print(f"✅ Features shape: {X.shape}")
print(f"✅ Target shape: {y.shape}")
print(f"\nFitur yang digunakan: {list(X.columns)}")

### 4.2 Label Encoding

In [ ]:
# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("✅ Label Encoding:")
for label, encoded in zip(le.classes_, le.transform(le.classes_)):
    print(f"   {label} -> {encoded}")

### 4.3 Train-Test Split

In [ ]:
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"✅ Data berhasil di-split!")
print(f"   Training set: {X_train.shape[0]} sampel ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"   Testing set: {X_test.shape[0]} sampel ({X_test.shape[0]/len(X)*100:.0f}%)")

In [ ]:
# Cek distribusi target setelah split
print("\nDistribusi target setelah split:")
print(f"\nTraining set:")
unique, counts = np.unique(y_train, return_counts=True)
for u, c in zip(unique, counts):
    print(f"   {le.classes_[u]}: {c} ({c/len(y_train)*100:.1f}%)")

print(f"\nTesting set:")
unique, counts = np.unique(y_test, return_counts=True)
for u, c in zip(unique, counts):
    print(f"   {le.classes_[u]}: {c} ({c/len(y_test)*100:.1f}%)")

### 4.4 Feature Scaling

In [ ]:
# Standardisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Data berhasil di-scale dengan StandardScaler!")
print(f"\nStatistik setelah scaling (training set):")
print(f"   Mean: {X_train_scaled.mean(axis=0).round(4)}")
print(f"   Std: {X_train_scaled.std(axis=0).round(4)}")

---
## 5. Model Training

### 5.1 Inisialisasi Model

In [ ]:
# Definisikan model-model yang akan digunakan
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial'),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
}

print(f"✅ {len(models)} model siap dilatih:")
for name in models.keys():
    print(f"   - {name}")

### 5.2 Training dan Evaluasi Model

In [ ]:
# Dictionary untuk menyimpan hasil
results = {}

print("🚀 Training Models...")
print("=" * 60)

for name, model in models.items():
    print(f"\n📌 {name}")
    print("-" * 40)
    
    # Training
    model.fit(X_train_scaled, y_train)
    
    # Prediksi
    y_pred = model.predict(X_test_scaled)
    
    # Evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    # Simpan hasil
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    
    print(f"   Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

print("\n" + "=" * 60)
print("✅ Training selesai!")

### 5.3 Perbandingan Model

In [ ]:
# Buat dataframe perbandingan
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test Accuracy': [r['accuracy'] for r in results.values()],
    'CV Accuracy (Mean)': [r['cv_mean'] for r in results.values()],
    'CV Accuracy (Std)': [r['cv_std'] for r in results.values()]
}).sort_values('Test Accuracy', ascending=False)

print("📊 Perbandingan Performa Model:")
comparison_df.round(4)

In [ ]:
# Visualisasi perbandingan
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(models))
width = 0.35

test_acc = [results[m]['accuracy'] for m in models.keys()]
cv_acc = [results[m]['cv_mean'] for m in models.keys()]

bars1 = ax.bar(x - width/2, test_acc, width, label='Test Accuracy', color='#3498db', edgecolor='black')
bars2 = ax.bar(x + width/2, cv_acc, width, label='CV Accuracy', color='#2ecc71', edgecolor='black')

ax.set_ylabel('Accuracy')
ax.set_title('Perbandingan Performa Model', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models.keys())
ax.legend()
ax.set_ylim(0, 1)

# Tambah label
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{height:.3f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{height:.3f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 6. Evaluasi Model Terbaik

In [ ]:
# Tentukan model terbaik
best_model_name = max(results.keys(), key=lambda k: results[k]['accuracy'])
best_result = results[best_model_name]

print(f"🏆 Model Terbaik: {best_model_name}")
print(f"   Accuracy: {best_result['accuracy']:.4f} ({best_result['accuracy']*100:.2f}%)")

### 6.1 Classification Report

In [ ]:
# Classification Report
print(f"📋 Classification Report - {best_model_name}")
print("=" * 60)
print(classification_report(y_test, best_result['predictions'], target_names=le.classes_))

### 6.2 Confusion Matrix

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, best_result['predictions'])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix - Counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[0],
            annot_kws={'size': 14})
axes[0].set_title(f'Confusion Matrix - {best_model_name}\n(Counts)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Confusion Matrix - Normalized
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Greens', 
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[1],
            annot_kws={'size': 12})
axes[1].set_title(f'Confusion Matrix - {best_model_name}\n(Normalized)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

### 6.3 Feature Importance

In [ ]:
# Feature Importance
print("📊 Feature Importance:")
print("=" * 50)

# Ambil feature importance dari Random Forest
rf_model = results['Random Forest']['model']
importance = rf_model.feature_importances_

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values('Importance', ascending=False)

print(feature_importance.to_string(index=False))

In [ ]:
# Visualisasi Feature Importance
plt.figure(figsize=(10, 6))

colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(feature_importance)))
bars = plt.barh(feature_importance['Feature'], feature_importance['Importance'], 
                color=colors, edgecolor='black')

plt.xlabel('Importance Score')
plt.title('Feature Importance (Random Forest)', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()

# Tambah nilai di samping bar
for bar, imp in zip(bars, feature_importance['Importance']):
    plt.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2, 
             f'{imp:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 7. Prediksi Data Baru

In [ ]:
# Contoh data baru untuk prediksi
data_baru = pd.DataFrame({
    'usia_jembatan': [15, 45, 30, 55, 10],
    'panjang': [50, 100, 75, 150, 30],
    'lebar': [10, 15, 12, 20, 8],
    'beban_harian': [500, 4000, 2000, 4500, 300],
    'curah_hujan': [1500, 2000, 1800, 2500, 1000],
    'tingkat_korosi': [2, 8, 5, 9, 1],
    'retakan': [5, 35, 18, 45, 3],
    'getaran_max': [0.5, 4.0, 2.5, 4.5, 0.3]
})

print("📝 Data Baru untuk Prediksi:")
data_baru

In [ ]:
# Preprocessing dan prediksi
data_baru_scaled = scaler.transform(data_baru)

# Prediksi dengan model terbaik
best_model = best_result['model']
prediksi = best_model.predict(data_baru_scaled)
prediksi_label = le.inverse_transform(prediksi)

# Prediksi probabilitas
if hasattr(best_model, 'predict_proba'):
    probabilities = best_model.predict_proba(data_baru_scaled)
else:
    probabilities = None

print(f"🎯 Hasil Prediksi dengan {best_model_name}:")
print("=" * 60)
for i in range(len(data_baru)):
    print(f"\nJembatan {i+1}:")
    print(f"   Kondisi: {prediksi_label[i]}")
    if probabilities is not None:
        print(f"   Probabilitas:")
        for j, cls in enumerate(le.classes_):
            print(f"      - {cls}: {probabilities[i][j]:.2%}")

In [ ]:
# Tambahkan hasil prediksi ke dataframe
hasil_prediksi = data_baru.copy()
hasil_prediksi['Prediksi Kondisi'] = prediksi_label

if probabilities is not None:
    for j, cls in enumerate(le.classes_):
        hasil_prediksi[f'Prob_{cls}'] = probabilities[:, j].round(3)

print("📊 Tabel Hasil Prediksi:")
hasil_prediksi

---
## 8. Ringkasan & Kesimpulan

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║                    📊 RINGKASAN PROYEK                       ║
╚══════════════════════════════════════════════════════════════╝
""")

print(f"""
📁 DATASET:
   • Total sampel: {len(df)}
   • Jumlah fitur: {len(X.columns)}
   • Fitur: {', '.join(X.columns)}
   • Target: 3 kelas (Baik, Sedang, Buruk)
   • Missing values: 0

🔧 PREPROCESSING:
   • Label Encoding untuk target
   • StandardScaler untuk fitur
   • Train-Test Split: 80% - 20%
   • Stratified split untuk menjaga distribusi kelas

🤖 MODEL YANG DIUJI:
""")

for name, result in results.items():
    print(f"   • {name}: {result['accuracy']*100:.2f}%")

print(f"""
🏆 MODEL TERBAIK: {best_model_name}
   • Test Accuracy: {best_result['accuracy']:.4f} ({best_result['accuracy']*100:.2f}%)
   • CV Accuracy: {best_result['cv_mean']:.4f} (+/- {best_result['cv_std']*2:.4f})

📈 FITUR TERPENTING:
""")

for _, row in feature_importance.head(3).iterrows():
    print(f"   • {row['Feature']}: {row['Importance']:.4f}")

print("""
══════════════════════════════════════════════════════════════
""")

In [ ]:
# Simpan model terbaik (opsional)
import joblib

joblib.dump(best_model, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(le, 'label_encoder.pkl')

print("✅ Model dan preprocessor berhasil disimpan!")
print("   • best_model.pkl")
print("   • scaler.pkl")
print("   • label_encoder.pkl")

---
## 🎉 Selesai!

Notebook ini mencakup:
1. ✅ Pembuatan dataset sintesis
2. ✅ EDA lengkap dengan visualisasi
3. ✅ Preprocessing data
4. ✅ Training multiple models
5. ✅ Evaluasi dan perbandingan model
6. ✅ Prediksi data baru
7. ✅ Export model